# Scope of project

After being satisfied that this is indeed possible now I'll lay out what needs to be done.

## Grading Rubric

1. Problem statement shown 1
2. <strike>Create tables containing valid sizes and spacings on a separate sheet and use these tables using vlookup. 1</strike>
3. Use functions like min, max, and if to automate design validation. 1.5
4. Formulas are used for calculations instead of manual entry of values 2
5. The equations entered into the spreadsheet are correct 1
6. Correct values are used from the problem statement, i.e. member dimensions, spans, etc. 1
7. Correct allowable values are used for comparison.  1
8. All the allowable stress adjustment values are correct and used. 1.5
9. Use of named cells to make formulas more readable 1 - **Extra Credit**

## Validation Techniques

1. Equations
  * Excel equations can be converted to symbolic equations for evaluation
    * Look up the variable name referenced in the cell to the left of the referenced value
      * This will need to be done with the aid of the tokenizer to find cell references
      * If cells are named (extra credit) then the cell reference will have to be have to be tracked down
    * Have the accurate symbolic equation already set using sympy
    * Subtract the two equations and simplify (correct should be equal to 0)
2. Manually entered data
  * These values will simply need to be checked against known values
3. Conditional statements
  * These should be a simple verification using the tokenizer
    * It may be necessary to validate the logic.


# Steps

## Document Cells that Need to be Examined

In [1]:
from collections import namedtuple
from collections import OrderedDict as Dict
Data = namedtuple("Data","description variable value equation condition")

data_cells = Dict([("C23" , Data('Species',     None,     'Southern Pine',
                               None,     None)),
                  ("C24" , Data('Grade',  None,     'No. 2 Dense', None,
                                None)),
                  ("C25" , Data('Moisture service conditions',     None,
                                'Wet',    None,     None)),
                  ("C26" , Data('Nominal Beam Thickness',     None,     4,
                              None,     None)),
                  ("C27" , Data('Nominal Beam Depth',    None,     10,  None,
                                None)),
                  ("C28" , Data(None,     'L', 12,  None,     None)),
                  ("C29" , Data(None,     's', 18,  None,     None)),
                  ("C30" , Data(None,     'wD',     25,  None,     None)),
                  ("C31" , Data(None,     'wL',     125, None,     None)),
                  ("C32" , Data(None,     'w', 150, 'wD+wL',  None)),
                  ("C35" , Data(None,     'b', 3.5, None,     None)),
                  ("C36" , Data(None,     'd', 9.25,     None,     None)),
                  ("C37" , Data(None,     'A', 32.38,    'b*d',    None)),
                  ("C38" , Data(None,     'S', 49.91,    '1/6*b*d**2',  
                                None)),
                  ("C39" , Data(None,     'I', 230.8,    '1/12*b*d**3', 
                                None)),
                  ("C43" , Data(None,     'Fb',     850, None,     None)),
                  ("C44" , Data(None,     'Fv',     175, None,     None)),
                  ("C45" , Data(None,     'E', 1600000,  None,     None)),
                  ("C51" , Data(None,     'CD',     1.25,     None,     
                                None)),
                  ("C52" , Data(None,     'CMb',    1,   None,     None)),
                  ("C53" , Data(None,     'Ct',     1,   None,     None)),
                  ("C54" , Data(None,     'CL',     1,   None,     None)),
                  ("C55" , Data(None,     'CF',     1,   None,     None)),
                  ("C56" , Data(None,     'Cfu',    1,   None,     None)),
                  ("C57" , Data(None,     'Ci',     1,   None,     None)),
                  ("C58" , Data(None,     'Cr',     1.15,     None,
                                'IF(s<=24, 1.15, 1)')),
                  ("C61" , Data(None,     'Fb_',    None,
                                'Fb*CD*CMb*Ct*CL*CF*Cfu*Ci*Cr',    None)),
                  ("C64" , Data(None,     'M', None,     'w*L**2/8',    
                                None)),
                  ("C66" , Data(None,     'fb',     None,     'M/S',    
                                None)),
                  ("C68" , Data('Bending Check',    None,     None,     None,
                                'IF(Fb_>=fb,"Good","Not Good")')),
                  ("C74" , Data(None,     'CD',     1.25,     'CD',     
                                None)),
                  ("C75" , Data(None,     'CMv',    0.97,     None,     
                                None)),
                  ("C76" , Data(None,     'Ct',     1,   'Ct',     None)),
                  ("C77" , Data(None,     'Ci',     1,   'Ci',     None)),
                  ("C80" , Data(None,     'Fv_',    None,
                                'Fv*CD*CMv*Ct*Ci',  None)),
                  ("C83" , Data(None,     'V', None,     'w*L/2',  None)),
                  ("C85" , Data(None,     'fv',     None,     '3/2*V/A',
                                None)),
                  ("C87" , Data('Shear Check', None,     None,     None,
                                'IF(Fv_>=fv,"Good","Not Good")')),
                  ("C94" , Data(None,     'CME',    0.9, None,     None)),
                  ("C95" , Data(None,     'Ct',     1,   'Ct',     None)),
                  ("C96" , Data(None,     'Ci',     1,   'Ci',     None)),
                  ("C100", Data(None,     'E_',     None,     'E*CME',  
                                None)),
                  ("C104", Data(None,     'L_360',  None,     'L/360',  
                                None)),
                  ("C107", Data(None,     'D_live', None,
                                '5*wL*L**4/(384*E_*I)*12**3', None)),
                  ("C109", Data('Live Deflection Check', None,     None,
                                None,     
                                'IF(L_360>=D_live,"Good","Not Good")')),
                  ("C112", Data(None,     'L_240',  None,     'L/240',  
                                None)),
                  ("C116", Data(None,     'D_total',     None,
                                '5*w*L**4/(384*E_*I)*12**3',  None)),
                  ("C118", Data('Total Deflection Check',     None,     None,
                                None,     
                                'IF(L_360>=D_total,"Good","Not Good")')),
                  ("C125", Data('Bending Summary',  None,     None,     'C68',
                               None)),
                  ("C126", Data('Shear Summary',    None,     None,     'C87',
                               None)),
                  ("C127", Data('Live Deflection Summary',    None,     None,
                                'C109',   None)),
                  ("C128", Data('Total Deflection Summary',   None,     None,
                                'C118',   None))]
)


## Load Spreadsheet

In [2]:
from openpyxl import load_workbook
wb = load_workbook('Timber Beam Template.xlsx')
sheets = wb.get_sheet_names()
ws = wb[sheets[0]]

### Get a list of all named Cells

In [3]:
named_ranges = wb.get_named_ranges()
names = [nr.name for nr in named_ranges]
print(names)

['C_D', 'C_F', 'C_Fu', 'C_F_F_B', 'C_i', 'C_L', 'C_M', 'C_r', 'C_t', 'E', 'F_b', 'F_v', 'WIDTH']


### Develop a Substitution List 

In [4]:
name_dict = dict()
CellID = namedtuple("CellID","variable cell")
for name in names:
    named_range = wb.get_named_range(name)
#     print(named_range)
    # ws.cell(nr7)
    named_range_cell = named_range.destinations[0][1]
    named_range_cell = named_range_cell.replace("$",'')
    equivalent = data_cells[named_range_cell].variable
    name_dict[name] = CellID(equivalent,named_range_cell)
#     print("{}: {}".format(name,equivalent))
#     cell_nr7 = ws.cell(nr7_cell)
#     cell_nr7.value
name_dict

{'C_D': CellID(variable='CD', cell='C51'),
 'C_F': CellID(variable='CF', cell='C55'),
 'C_F_F_B': CellID(variable='CF', cell='C55'),
 'C_Fu': CellID(variable='Cfu', cell='C56'),
 'C_L': CellID(variable='CL', cell='C54'),
 'C_M': CellID(variable='CMb', cell='C52'),
 'C_i': CellID(variable='Ci', cell='C57'),
 'C_r': CellID(variable='Cr', cell='C58'),
 'C_t': CellID(variable='Ct', cell='C53'),
 'E': CellID(variable='E', cell='C45'),
 'F_b': CellID(variable='Fb', cell='C43'),
 'F_v': CellID(variable='Fv', cell='C44'),
 'WIDTH': CellID(variable='d', cell='C36')}

## Setup Sympy Variables

In [5]:
from sympy import symbols, simplify
variables = []
variable_dict = dict()
for cell, content in data_cells.items():
    if content.variable != None:
        variable_dict[content.variable] = content.value
        exec("{} = symbols('{}')".format(content.variable,content.variable))

### Truth Table Evaluation
This is used for verifying IF statements.

In [6]:
import numpy as np
def truth(symbols, comparison):
    s1, s2 = symbols
    check = []
    check.append(eval(comparison.replace(s1,'1').replace(s2,'1')))
    check.append(eval(comparison.replace(s1,'0').replace(s2,'1')))
    check.append(eval(comparison.replace(s1,'0').replace(s2,'0')))
    check.append(eval(comparison.replace(s1,'1').replace(s2,'0')))
    return check
A_ = truth(['x','y'],'x>=y')
B_ = truth(['x','y'],'x<y')
print(A_)
print(B_)
print(np.array(A_) & np.array(B_))
print(np.array(A_) & ~ np.array(B_))

[True, False, True, True]
[False, True, False, False]
[False False False False]
[ True False  True  True]


## Examine each Cell's Content

### Load Libraries

### Text Entry Checks

In [7]:
def text_only(ID, CELL, content, cell):
    "Used to verify text only entries"
    description, variable, value, equation, condition = CELL
    if content.lower().strip() == value.lower().strip():
        print("{} is Correct: ".format(ID),end='')
        print("Text entry for {} = {}".format(cell,content))
        return 1
    else:
        print(ID)
        print('\tText entry ("{}") '.format(content),end='')
        print('does not match the answer of "{}"'.format(value))
        return 0

### Numeric Enrtry Checks

In [8]:
def number_only(ID, CELL, content, cell):
    "Checks the solution if the student only provided a number for an answer"
    description, variable, value, equation, condition = CELL
    if content == value:
        print("{} is right, ({} = {})".format(ID, variable, content))
        return 1
    else:
        print(ID)
#         print('\t{} in {} is wrong'.format(content,cell))
        if equation != None:
            print('\tThe answer should be an equation '
                  'like "{}" not {}'.format(equation, content))
        elif condition != None:
            print('\tThe answer should be a conditional statement '
                  'like "{}" not {}'.format(condition, content))
        else:
            des = description if variable == None else variable
            print('\tThe value for {} should be {} not {}'.format(des, value,
                                                                 content))
        return -1
           

### If-Then Statement Checks

In [9]:
import re

comparison = re.compile(r'(?:IF\()(.+?)(?:,)')
T_and_F = re.compile(r'(?:\b,)(.+?)(?:,)(.+?)(?:\))')

def conditional_check(ID, CELL, content, cell, tokens, variables):
    "Checks if-then answers"
    description, variable, value, equation, condition = CELL
    if "IF" not in content:
        print(ID)
        print('\t{} in {} is wrong'.format(content,cell))
        print('\tThe answer should be a conditional statement '
              'like "{}"'.format(condition))
        return -1
    else:
        Condition = ''.join(tokens)
        compare = comparison.findall(Condition)
        student = np.array(truth(variables, compare[0]))
        my_compare = comparison.findall(CELL.condition)
        answer = np.array(truth(variables, my_compare[0]))
        info = "\t{} is correct if the following is true".format(cell)
        if all(student==answer):
            s_choices = T_and_F.findall(Condition)[0]
            a_choices = T_and_F.findall(CELL.condition)[0]
        elif all(student==~answer):
            s_choices = T_and_F.findall(Condition)[0][::-1]
            a_choices = T_and_F.findall(CELL.condition)[0] 
        else:
            # IF statement is wrong (the condition cannot match)
            print(ID)
            print("\tStudent function is at {} = {}".format(cell,Condition))
            print('\t{} is wrong'.format(cell))
            return -1
        manual = False
        for a_choice,s_choice in zip(a_choices,s_choices):
            if s_choice.lower().strip() != a_choice.lower().strip():
                manual = True
                info += '\n\t{} means {}'.format(s_choice,a_choice)
        if manual:
            print(ID)
            print("\tStudent function is at {} = {}".format(cell,Condition))
            print(info)
            return 0
        else:
            print("{}: {} is correct.".format(ID, cell))
            return 1

### Equation Checks

In [10]:
def equation_check(ID, CELL, content, cell, tokens, variables):
    "This verifies student equations are correct (Excel functions not used)"
    description, variable, value, equation, condition = CELL
    if len(variables)!=0:
        Equation = ''.join(tokens)
        out = "Student equation at {} = {}".format(cell, Equation)
        check = "{}-({})".format(Equation, equation)
        results = simplify(check)
        if results == 0:
            out +=' - is Correct'
            out = ID + ': '+out
            val = 1
        else:
            print(ID)
            out +=' - is Wrong!'
            val = -1
        print(out)
    else:
        print(ID)
        print('\t{} in {} is wrong'.format(content,cell))
        print('\tThe answer should be an equation '
              'like "{}"'.format(equation))
        val = -1
    return val

### Parsing for Non-Text and Numeric Cells

In [11]:
from openpyxl.formula import Tokenizer

def parse(entry):
    "Parses student entry"
    tok = Tokenizer(entry)
    tok.parse()
    tokens = []
    variables = []
    for token in tok.items:
        if token.subtype=='RANGE':
            # This is a cell reference like C38, 
            # convert it to a symbolic variable
            ref_cell = token.value
            if ref_cell not in data_cells:
                # This is a named range and it needs to be a variable
                tokens.append(name_dict[ref_cell].variable)
                variables.append(name_dict[ref_cell].variable)
            elif data_cells[ref_cell].variable != None:
                # This cell does not have a symbolic representation
                tokens.append(data_cells[ref_cell].variable)
                variables.append(data_cells[ref_cell].variable)
            else:
                tokens.append(ref_cell)
        elif token.value=='^':
            # Converting for python notation
            tokens.append('**')
        else:
            tokens.append(token.value)
    Tokens = namedtuple('Tokens','tokens variables')
#     print("\tVariables = {}, tokens = {}".format(variables, tokens))
    return Tokens(tokens, variables)

### Scoring

In [12]:
import numbers

def CHECK():
    num = []
    equ = []
    cond = []
    txt = []
    for cell, content in data_cells.items():
        ws_cell = ws.cell(cell)
        CELL = data_cells[cell]
        des = CELL.description if CELL.variable==None else CELL.variable
        ID = "Cell {} ({})".format(cell, des)
    #     _description, _variable, _value, _equation, _condition = CELL
        if isinstance(ws_cell.value ,numbers.Number):
            ID = 'Numeric: '+ID
            # The student only entered a number in this cell
            num.append(number_only(ID, CELL, ws_cell.value, cell))
        elif (CELL.variable==None and CELL.equation==None and 
              CELL.condition==None and CELL.description!=None 
              and CELL.value!=None):
            ID = 'Text: '+ID
            # Text only solution
            txt.append(text_only(ID, CELL, ws_cell.value, cell))
        else:
            tokens, variables = parse(ws_cell.value)
            if CELL.condition != None:
                ID = 'Conditional: '+ID
                cond.append(conditional_check(ID, CELL, ws_cell.value, cell, 
                                              tokens, variables))
            elif CELL.equation != None:
                ID = 'Equation: '+ID
                equ.append(equation_check(ID, CELL, ws_cell.value, cell, tokens, 
                               variables))
    Results = namedtuple('Results','numeric equation condition text')
    RightWrong = namedtuple('RightWrong','right wrong manual_verification')
    results = Results(RightWrong(num.count(1),num.count(-1),num.count(0)),
                      RightWrong(equ.count(1),equ.count(-1),equ.count(0)),
                      RightWrong(cond.count(1),cond.count(-1),cond.count(0)),
                      RightWrong(txt.count(1),txt.count(-1),txt.count(0)))
    return results

In [13]:
Results = CHECK()

Text: Cell C23 (Species)
	Text entry ("Not Tree") does not match the answer of "Southern Pine"
Text: Cell C24 (Grade)
	Text entry ("Not B+") does not match the answer of "No. 2 Dense"
Text: Cell C25 (Moisture service conditions)
	Text entry ("Wet or Dry") does not match the answer of "Wet"
Numeric: Cell C26 (Nominal Beam Thickness)
	The value for Nominal Beam Thickness should be 4 not 1
Numeric: Cell C27 (Nominal Beam Depth)
	The value for Nominal Beam Depth should be 10 not 1
Numeric: Cell C28 (L)
	The value for L should be 12 not 1
Numeric: Cell C29 (s)
	The value for s should be 18 not 1
Numeric: Cell C30 (wD)
	The value for wD should be 25 not 1
Numeric: Cell C31 (wL)
	The value for wL should be 125 not 1
Numeric: Cell C32 (w)
	The answer should be an equation like "wD+wL" not 1
Numeric: Cell C35 (b) is right, (b = 3.5)
Numeric: Cell C36 (d) is right, (d = 9.25)
Numeric: Cell C37 (A)
	The answer should be an equation like "b*d" not 1
Numeric: Cell C38 (S)
	The answer should be an e

In [14]:
print('{:<15} {}'.format("Numeric",Results.numeric))
print('{:<15} {}'.format('Equation',Results.equation))
print('{:<15} {}'.format('Condition',Results.condition))
print('{:<15} {}'.format('Text',Results.text))

Numeric         RightWrong(right=11, wrong=28, manual_verification=0)
Equation        RightWrong(right=1, wrong=4, manual_verification=0)
Condition       RightWrong(right=1, wrong=3, manual_verification=0)
Text            RightWrong(right=0, wrong=0, manual_verification=3)


# Original Check

In [15]:
from openpyxl.formula import Tokenizer
import re

comparison = re.compile(r'(?:IF\()(.+?)(?:,)')
T_and_F = re.compile(r'(?:\b,)(.+?)(?:,)(.+?)(?:\))')

for cell, content in data_cells.items():
    ws_cell = ws.cell(cell)
    CELL = data_cells[cell]
    try:
        tok = Tokenizer(ws_cell.value)
        tok.parse()
        equation = []
        is_equation = False
        function = False
        nothing = False
        if len(tok.items)==0:
            # There is nothing or a zero in the cell
            nothing = True
        else:
            comparison_variables = [] # (in case an IF statement is found)
        for token in tok.items:
            if token.type == 'FUNC':
                # This should be an IF statement
                function = True
            if token.subtype=='RANGE':
                # This is a cell reference like C38, 
                # convert it to a symbolic variable
                is_equation = True
                ref_cell = token.value
                if ref_cell not in data_cells:
                    # This is a named range and it needs to be a variable
                    cell_variable = name_dict[ref_cell].variable
                else:
                    # This cell does not have a symbolic representation
                    cell_variable = data_cells[ref_cell].variable
                    if cell_variable == None:
                        cell_variable = ref_cell
                equation.append(cell_variable)
                if function:
                    # Keep track of variable bing compared in IF statements
                    comparison_variables.append(cell_variable)
            elif token.value=='^':
                # Converting for python notation
                equation.append('**')
            else:
                equation.append(token.value)
        Equation = ''.join(equation)
        if not function and not nothing:
            if is_equation:
                out = "Student equation at {} = {}".format(cell, Equation)
                check = "{}-({})".format(Equation, CELL.equation)
                results = simplify(check)
                if results == 0:
                    out +=' - Equation is good'
                else:
                    out +=' - Equation is BAD!'
                print(out)
            else:
                print("Text entry in cell {} = {}".format(cell,Equation))
                if (CELL.equation !=None or CELL.variable !=None or 
                    CELL.condition !=None):
                    print("\tWrong: This should not be a text entry.")
                else:
                    print('The answer key has "{}"'.format(CELL.value))
        elif not nothing:
            if CELL.condition != None:
                # Validate IF statement equivalence
                print("Student function is at {} = {}".format(cell,Equation))
                compare = comparison.findall(Equation)
                student = np.array(truth(comparison_variables, compare[0]))
                my_compare = comparison.findall(CELL.condition)
                answer = np.array(truth(comparison_variables, my_compare[0]))
                info = "\t{} is correct if the following is true".format(cell)
                if all(student==answer):
                    s_choices = T_and_F.findall(Equation)[0]
                    a_choices = T_and_F.findall(CELL.condition)[0]
                    print(info)
                    for a_choice,s_choice in zip(a_choices,s_choices):
                        print('\t{} means {}'.format(s_choice,a_choice))
                elif all(student==~answer):
                    s_choices = T_and_F.findall(Equation)[0][::-1]
                    a_choices = T_and_F.findall(CELL.condition)[0]
                    print(info)
                    for a_choice,s_choice in zip(a_choices,s_choices):
                        print('\t{} means {}'.format(s_choice,a_choice))
                else:
                    # IF statement is wrong
                    print('\t{} is wrong'.format(cell))
            else:
                if CELL.description != None:
                    desc = CELL.description  
                else:
                    desc = CELL.variable
                print("\tNo Condition found for {}".format(desc))
                print("\tStudent used {}".format(Equation))
        else:
            if CELL.value != 0:
                print("\t{} is wrong".format(cell))
            else:
                print("{} = 0 - Correct".format(cell))
    except TypeError:
        if ws_cell.value == content.value:
            print("{} is right, ({} = {})".format(cell,CELL.variable,
                                                  ws_cell.value))
        else:
            print('\t{} in {} is wrong'.format(ws_cell.value,cell))


Text entry in cell C23 = Not Tree
The answer key has "Southern Pine"
Text entry in cell C24 = Not B+
The answer key has "No. 2 Dense"
Text entry in cell C25 = Wet or Dry
The answer key has "Wet"
	1 in C26 is wrong
	1 in C27 is wrong
	1 in C28 is wrong
	1 in C29 is wrong
	1 in C30 is wrong
	1 in C31 is wrong
	1 in C32 is wrong
C35 is right, (b = 3.5)
C36 is right, (d = 9.25)
	1 in C37 is wrong
	1 in C38 is wrong
Student equation at C39 = 1/12*(b*d**3) - Equation is good
	C43 is wrong
	C44 is wrong
	C45 is wrong
	1 in C51 is wrong
	No Condition found for CMb
	Student used IF(C25="Wet",0.85,1)
C53 is right, (Ct = 1)
C54 is right, (CL = 1)
C55 is right, (CF = 1)
C56 is right, (Cfu = 1)
C57 is right, (Ci = 1)
	1 in C58 is wrong
	C61 is wrong
	C64 is wrong
	C66 is wrong
Student function is at C68 = IF(Fb_>=fb,"Good","Not Good")
	C68 is correct if the following is true
	"Good" means "Good"
	"Not Good" means "Not Good"
	1 in C74 is wrong
	1 in C75 is wrong
C76 is right, (Ct = 1)
C77 is right, 

# Appendix: Transform data_cells Content

This was only used in the development.

In [16]:
# txt = """data_cells = {'C23':Data('Species', None,'Southern Pine',None,None),
#               'C24':Data('Grade', None,'No. 2 Dense',None,None),
#               'C25':Data('Moisture service conditions', None,'Wet',None,None),
#               'C26':Data('Nominal Beam Thickness', None,4,None,None),
#               'C27':Data('Nominal Beam Depth', None,10,None,None),
#               'C28':Data(None, 'L',12,None,None),
#               'C29':Data(None, 's',18,None,None),
#               'C30':Data(None, 'wD',25,None,None),
#               'C31':Data(None, 'wL',125,None,None),
#               'C35':Data(None, 'b',3.5,None,None),
#               'C36':Data(None, 'd',9.25,None,None),
#               'C43':Data(None, 'Fb',850,None,None),
#               'C44':Data(None, 'Fv',175,None,None),
#               'C45':Data(None, 'E',1600000,None,None),
#               'C51':Data(None, 'CD',1.25,None,None),
#               'C52':Data(None, 'CMb',1,None,None),
#               'C53':Data(None, 'Ct',1,None,None),
#               'C54':Data(None, 'CL',1,None,None),
#               'C55':Data(None, 'CF',1,None,None),
#               'C56':Data(None, 'Cfu',1,None,None),
#               'C57':Data(None, 'Ci',1,None,None),
#               'C75':Data(None, 'CMv',0.97,None,None),
#               'C94':Data(None, 'CME',0.9,None,None),
#               'C32':Data(None, 'w',150,'wD+wL',None),
#               'C37':Data(None, 'A',32.38,'b*d',None),
#               'C38':Data(None, 'S',49.91,'1/6*b*d**2',None),
#               'C39':Data(None, 'I',230.8,'1/12*b*d**3',None),
#               'C58':Data(None, 'Cr',1.15,None,'IF(s<=24,1.15,1)'),
#               'C61':Data(None, 'Fb_', None, 'Fb*CD*CMb*Ct*CL*CF*Cfu*Ci*Cr',None),
#               'C64':Data(None, 'M', None, 'w*L**2/8',None),
#               'C66':Data(None, 'fb', None, 'M/S',None),
#               'C68':Data('Bending Check', None, None, None,'IF(Fb_>=fb,"Good","Not Good")'),
#               'C74':Data(None, 'CD',1.25, 'CD',None),
#               'C76':Data(None, 'Ct',1, 'Ct',None),
#               'C77':Data(None, 'Ci',1, 'Ci',None),
#               'C80':Data(None, 'Fv_', None, 'Fv*CD*CMv*Ct*Ci',None),
#               'C83':Data(None, 'V', None, 'w*L/2',None),
#               'C85':Data(None, 'fv', None, '3/2*V/A',None),
#               'C87':Data('Shear Check', None, None, None,'IF(Fv_>=fv,"Good","Not Good")'),
#               'C95':Data(None, 'Ct',1, 'Ct',None),
#               'C96':Data(None, 'Ci',1, 'Ci',None),
#               'C100':Data(None, 'E_', None, None,None),
#               'C104':Data(None, 'L_360', None, 'L/360',None),
#               'C107':Data(None, 'D_live', None, '5*wL*L**4/(384*E_*I)*12**3',None),
#               'C109':Data('Live Deflection Check', None, None, None,'IF(L_360>=D_live,"Good","Not Good")'),
#               'C112':Data(None, 'L_240', None, 'L/240',None),
#               'C116':Data(None, 'D_total', None, '5*w*L**4/(384*E_*I)*12**3',None),
#               'C118':Data('Total Deflection Check', None, None, None,'IF(L_360>=D_total,"Good","Not Good")'),
#               'C125':Data('Bending Summary', None, None, 'C68',None),
#               'C126':Data('Shear Summary', None, None, 'C87',None),
#               'C127':Data('Live Deflection Summary', None, None, 'C109',None),
#               'C128':Data('Total Deflection Summary', None,None,'C118',None),
# }"""
# import re
# comma = re.compile(r',(?!\s)')
# out_lines = []
# for line in txt.splitlines():
#     line = comma.sub(', ',line)
# #     if line!=Line:
# #         print(Line)
#     out_lines.append(line)
# print('\n'.join(out_lines))
# # cell_name = re.compile(r"(?:Data\()(\'\w\d+\')")
# # data = re.compile(r"Data\('.*'")
# # # cell_name.findall(txt)
# # for cell in data.findall(txt):
# #     txt = txt.replace(cell,"{},''".format(cell))
# # cell_data = re.compile(r'(?:Data\()(\'.*?\')(?:,)')
# # print(txt)
# # changes = list(set(cell_data.findall(txt)))
# # print(changes)
# # not_variables = ["'Species'","'Grade'"]
# # print(txt)
# # for change in changes:
# #     if 'None' in change:
# #         continue
# #     elif ' ' in change or change in not_variables:
# #         txt = txt.replace(change,"{}, None".format(change))
# #     else:
# #         txt = txt.replace(change,"None, {}".format(change))
# # txt = txt.replace("''","None")
# # print(txt)